<a href="https://colab.research.google.com/github/SheilaLiborio/controle_estoque_loja_games/blob/main/Projeto_Loja_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTS E BANCO

In [ ]:
import sqlite3
import time
import datetime
from datetime import date
from datetime import datetime
from IPython.display import clear_output
from rich.align import Align
from rich import print
from rich.table import Table, box
from rich.panel import Panel
from rich.text import Text
from rich.rule import Rule
from rich.prompt import Prompt, Confirm
from rich.console import Console
import datetime
from rich.live import Live
from rich.progress import Progress, SpinnerColumn, TextColumn, BarColumn
console = Console()


db = 'loja_games.db'

#TABELA

In [ ]:
def criar_tabelas_banco():
  try:
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()

      cursor.execute('DROP TABLE IF EXISTS fornecedores')
      cursor.execute('DROP TABLE IF EXISTS produtos')
      cursor.execute('DROP TABLE IF EXISTS reposicoes')
      cursor.execute('DROP TABLE IF EXISTS historico_precos')

      cursor.execute('PRAGMA foreign_keys = ON')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS fornecedores(
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          nome TEXT NOT NULL,
          cnpj TEXT NOT NULL UNIQUE,
          telefone INTEGER DEFAULT 0
        );
      ''')


      cursor.execute('''
        CREATE TABLE IF NOT EXISTS produtos(
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          nome TEXT UNIQUE NOT NULL,
          preco REAL NOT NULL,
          categoria TEXT NOT NULL,
          quantidade_estoque INTEGER NOT NULL,
          fornecedor_id INTEGER NOT NULL,
          FOREIGN KEY (fornecedor_id) REFERENCES fornecedores(id)
        );
      ''')


      cursor.execute('''
        CREATE TABLE IF NOT EXISTS reposicoes(
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          produto_id INTEGER not NULL,
          fornecedor_id INTEGER NOT NULL,
          quantidade INTEGER NOT NULL,
          data_reposicao DATE NOT NULL,
          FOREIGN KEY (produto_id) REFERENCES produtos(id),
          FOREIGN KEY (fornecedor_id) REFERENCES fornecedores(id)
        );
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS historico_precos (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          id_produto INTEGER NOT NULL,
          preco_antigo REAL NOT NULL,
          preco_novo REAL NOT NULL,
          data_alteracao DATE NOT NULL,
          FOREIGN KEY (id_produto) REFERENCES produtos(id)
        );
      ''')

    print('Tabelas criada com sucesso!')
  except sqlite3.Error as erro:
    print(f'Erro ao criar tabela: {erro}')

In [ ]:
def popular_tabelas_banco():
  try:
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()

      # Inserir dados na tabela fornecedores
      cursor.execute('INSERT INTO fornecedores (nome, cnpj, telefone) VALUES (?, ?, ?)', ('Sony', '12345678000190', 11987654321))
      cursor.execute('INSERT INTO fornecedores (nome, cnpj, telefone) VALUES (?, ?, ?)', ('Microsoft', '98765432000121', 21912345678))
      cursor.execute('INSERT INTO fornecedores (nome, cnpj, telefone) VALUES (?, ?, ?)', ('Nintendo', '45678901000154', 31998761234))

      # Inserir dados na tabela produtos
      cursor.execute('INSERT INTO produtos (nome, preco, categoria, quantidade_estoque, fornecedor_id) VALUES (?, ?, ?, ?, ?)', ('PlayStation 5', 4999.90, 'Console', 50, 1))
      cursor.execute('INSERT INTO produtos (nome, preco, categoria, quantidade_estoque, fornecedor_id) VALUES (?, ?, ?, ?, ?)', ('Xbox Series X', 4599.90, 'Console', 40, 2))
      cursor.execute('INSERT INTO produtos (nome, preco, categoria, quantidade_estoque, fornecedor_id) VALUES (?, ?, ?, ?, ?)', ('Nintendo Switch OLED', 2999.90, 'Console', 60, 3))
      cursor.execute('INSERT INTO produtos (nome, preco, categoria, quantidade_estoque, fornecedor_id) VALUES (?, ?, ?, ?, ?)', ('God of War Ragnarok', 299.90, 'Jogos', 100, 1))
      cursor.execute('INSERT INTO produtos (nome, preco, categoria, quantidade_estoque, fornecedor_id) VALUES (?, ?, ?, ?, ?)', ('Halo Infinite', 259.90, 'Jogos', 80, 2))
      cursor.execute('INSERT INTO produtos (nome, preco, categoria, quantidade_estoque, fornecedor_id) VALUES (?, ?, ?, ?, ?)', ('The Legend of Zelda: Tears of the Kingdom', 349.90, 'Jogos', 90, 3))

      # Inserir dados na tabela reposicoes
      cursor.execute('INSERT INTO reposicoes (produto_id, fornecedor_id, quantidade, data_reposicao) VALUES (?, ?, ?, ?)', (1, 1, 20, '2023-10-27'))
      cursor.execute('INSERT INTO reposicoes (produto_id, fornecedor_id, quantidade, data_reposicao) VALUES (?, ?, ?, ?)', (2, 2, 15, '2023-10-27'))
      cursor.execute('INSERT INTO reposicoes (produto_id, fornecedor_id, quantidade, data_reposicao) VALUES (?, ?, ?, ?)', (3, 3, 25, '2023-10-27'))

      conexao.commit()
    print('Tabelas populadas com sucesso!')
  except sqlite3.Error as erro:
    print(f'Erro ao popular tabelas: {erro}')


In [ ]:
criar_tabelas_banco()
popular_tabelas_banco()

Tabelas criada com sucesso!

Tabelas populadas com sucesso!

#VALIDAÇÕES

In [ ]:
def validar_data():
  while True:
    try:
      data = input("Digite a data(AAAA-MM-DD) ou pressione Enter para preencher com a data atual: ").strip()
      if data == "":
        return date.today()
      elif data[4] == "-" and data[7] == "-":
        return data
      else:
        print("[bright_red]Digite uma data valida[/]")
    except KeyboardInterrupt as erro:
      print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_estoque_produto_atualizar():
  estoque = int(input("Digite o estoque do produto: "))
  try:
    while True:
      if estoque >= 0:
        return estoque
      else:
          print("[bright_red]ERRO: Estoque não pode ser inferior a 0[/]")
          estoque = int(input("Digite o estoque do produto: "))
  except ValueError as erro:
    print(f"[bright_red]ERRO: Digite apenas números! -> {erro}[/]")
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_estoque_produto():
  estoque = int(input("Digite o estoque do produto: "))
  try:
    while True:
      if estoque > 0:
        return estoque
      else:
          print("[bright_red]ERRO: Estoque não pode ser igual ou inferior a 0[/]")
          estoque = int(input("Digite o estoque do produto: "))
  except ValueError as erro:
    print(f"[bright_red]ERRO: Digite apenas números! -> {erro}[/]")
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_estoque_reposicao():
  estoque = int(input("Digite a nova quantidade: "))
  try:
    while True:
      if estoque > 0:
        return estoque
      else:
          print("[bright_red]ERRO: A nova quantidade não pode ser igual ou inferior a 0[/]")
          estoque = int(input("Digite a nova quantidade: "))
  except ValueError as erro:
    print(f"[bright_red]ERRO: Digite apenas números! -> {erro}[/]")
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_preco_produto():
  preco = float(input("Digite o preço do produto: "))
  try:
    while True:
      if preco > 0:
        return preco
      else:
          print("[bright_red]ERRO: Preço não pode ser igual ou inferior a 0[/]")
          preco = float(input("Digite o preço do produto: "))
  except ValueError as erro:
    print(f"[bright_red]ERRO: Digite apenas números! -> {erro}[/]")
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_cnpj():
  try:
    cnpj = input("Digite o CNPJ do fornecedor(apenas números): ")
    while True:
      if cnpj.isdigit() == True and len(cnpj) == 14:
        return cnpj
      else:
        print("[bright_red]ERRO: CNPJ deve conter apenas números e 14 digitos[/]")
        cnpj = input("Digite o CNPJ do fornecedor(apenas números): ")
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_fornecedor_id():
  while True:
    fornecedor = int(input("Digite o ID do fornecedor: "))
    try:

      with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute(""" SELECT id FROM fornecedores WHERE id = ?""", (fornecedor, ))
        id_fornecedor = cursor.fetchone()

        if id_fornecedor:
            return fornecedor

        elif not id_fornecedor:
          print("[bright_red]ERRO: Digite o ID de um fornecedor existente![/]")
          time.sleep(2)

        else:
            print("[bright_red]ERRO: Digite o ID de um fornecedor existente![/]")
            time.sleep(2)

    except ValueError:
      print("[bright_red]Digite apenas números![/]")
      time.sleep(2)
    except KeyboardInterrupt as erro:
      print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')
      time.sleep(1)

In [ ]:
def validar_produto_id():
  while True:
    try:

      id = int(input("Digite o ID do produto: "))
      with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute(""" SELECT id FROM produtos WHERE id = ?""", (id, ))
        id_produto = cursor.fetchone()

        if id_produto:
            return id
        else:
            print("[bright_red]ERRO: Digite o ID de um produto existente![/]")
    except ValueError:
      print("[bright_red]Digite apenas números![/]")

    except KeyboardInterrupt as erro:
      print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_nome_id():
    try:
      while True:
        nome = input("Digite o nome do produto: ").strip().title()
        with sqlite3.connect(db) as conexao:
          cursor = conexao.cursor()
          cursor.execute(""" SELECT nome FROM produtos WHERE nome = ?""", (nome, ))
          nome_produto = cursor.fetchone()

          if not nome_produto:
            return nome
          else:
            console.print("[bright_red]ERRO: Este produto já consta no sistema![/]")
    except KeyboardInterrupt as erro:
      print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_reposicao_id():
  while True:
    try:

      id = int(input("Digite o ID da reposição: "))
      with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute(""" SELECT id FROM reposicoes WHERE id = ?""", (id, ))
        id_reposicao = cursor.fetchone()

        if id_reposicao:
            return id
        else:
            print("[bright_red]ERRO: Digite o ID de uma reposição existente![/]")
    except ValueError:
      print("[bright_red]Digite apenas números![/]")

    except KeyboardInterrupt as erro:
      print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def validar_estoque_nao_negativo(qtd_resultante: int) -> None:
    if qtd_resultante < 0:
        raise ValueError("Operação negada: o estoque não pode ficar negativo.")

In [ ]:
def validar_nome_produto_unico(nome: str, nomes_existentes: set[str]) -> None:
    if nome.lower() in (n.lower() for n in nomes_existentes):
        raise ValueError(f"O produto “{nome}” já existe no sistema.")

In [ ]:
def validar_quantidade_reposta(qtd: int) -> None:
    if qtd <= 0:
        raise ValueError("Quantidade de reposição deve ser maior que zero.")

In [ ]:
def verificar_alerta_estoque_baixo():
  try:
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()
      cursor.execute("SELECT COUNT(*) FROM produtos WHERE quantidade_estoque < 5")
      qtd = cursor.fetchone()[0]
      return qtd > 0
  except:
    return False

#INSERT'S

In [ ]:
def cadastrar_fornecedores():
  time.sleep(1)
  try:
      console.print(Rule(style="white"), width=120)

      nome = input("Digite o nome do fornecedor: ").strip().title()
      cnpj = validar_cnpj()
      telefone = input("Digite a telefone: ").strip()

      valores = (nome, cnpj, telefone)

      with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute("""
          INSERT INTO fornecedores(nome,cnpj,telefone)
          VALUES(?,?,?)

          """, valores)

      console.print(f"[bright_green]O FORNECEDOR {nome}, FOI INSERIDO COM SUCESSO![/]")
      time.sleep(3)
  except sqlite3.IntegrityError:
    print("[bright_red]ERRO: Este CNPJ ja consta no sistema![/]")
    time.sleep(3)

In [ ]:
def cadastrar_produtos():
  time.sleep(1)
  try:
      console.print(Rule(style="white"), width=120)

      nome = validar_nome_id()
      preco = validar_preco_produto()
      categoria = input("Digite a categoria: ").strip().title()
      estoque = validar_estoque_produto()
      fornecedor = validar_fornecedor_id()

      valores = (nome, preco, categoria, estoque, fornecedor)

      with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute("""
        INSERT INTO produtos(nome,preco,categoria,quantidade_estoque,fornecedor_id)
        VALUES(?,?,?,?,?)

          """, valores)

      console.print(f"[bright_green]PRODUTO {nome}, FOI INSERIDO COM SUCESSO![/]")
      time.sleep(3)
  except sqlite3.IntegrityError as erro:
      print(f"[bright_red]ERRO: Produto {nome}, já consta no sistema! -> {erro}[/]")
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def inserir_reposicao():
  time.sleep(1)
  try:
      console.print(Rule(style="white"), width=120)
      time.sleep(2)
      id = validar_produto_id()

      with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute("""
        SELECT * FROM produtos
        WHERE id = ?
        """, (id,))
        produto = cursor.fetchall()
        for prod in produto:
          console.print(f"Produto: [bright_yellow]{prod[1]}[/]")
          console.print(f"Estoque: {prod[4]}")

          time.sleep(2)

        fornecedor_id = validar_fornecedor_id()
        with sqlite3.connect(db) as conexao:
          cursor = conexao.cursor()
          cursor.execute("""
          SELECT * FROM fornecedores
          WHERE id = ?
          """, (fornecedor_id,))
        fornecedor = cursor.fetchall()
        for f in fornecedor:
          console.print(f"Fornecedor: [bright_yellow]{f[1]}[/]")

          time.sleep(2)

        quantidade = validar_estoque_produto()
        data = validar_data()

        if produto:
          with sqlite3.connect(db) as conexao:
            cursor = conexao.cursor()
            cursor.execute(" UPDATE produtos SET quantidade_estoque = quantidade_estoque + ? WHERE id = ?", (quantidade, id))
            cursor.execute(" INSERT INTO reposicoes(produto_id, fornecedor_id, quantidade, data_reposicao) VALUES(?,?,?,?) ", (id, fornecedor_id, quantidade, data))

            console.print(f"[bright_green]O ESTOQUE DO PRODUTO {prod[1]}, FOI ATUALIZADO COM SUCESSO![/]")
            time.sleep(3)
  except KeyboardInterrupt as erro:
      print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

#LISTAR

In [ ]:
def listar_produtos():
	try:
		clear_output(wait=True)
		exibir_cabecalho_sistema(titulo="🕹️ GERENCIAMENTO DE PRODUTOS 🎮")
		with sqlite3.connect(db) as conexao:
			cursor = conexao.cursor()
			cursor.execute("SELECT * FROM produtos")
			produtos = cursor.fetchall()
			if produtos:
				colunas = [
					("ID", "yellow", "center"),
					("Nome", "yellow", "left"),
					("Preço", "yellow", "right"),
					("Categoria", "yellow", "left"),
					("Quantidade", "yellow", "right"),
					("Fornecedor", "yellow", "left")
				]

				produtos_coloridos = []
				for p in produtos:
					linha = list(p)
					quantidade = p[4]  # Supondo que o 5º campo é a quantidade
					linha[4] = f"[bold red]{quantidade}[/bold red]" if quantidade < 5 else str(quantidade)
					produtos_coloridos.append(linha)

				exibir_tabela(colunas, produtos_coloridos)
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()
			else:
				console.print("Nenhum produto encontrado!", style="yellow")
				time.sleep(3)
	except sqlite3.Error as erro:
		console.print(f"Erro ao listar os produtos: {erro}", style="red")
		time.sleep(3)

In [ ]:
def listar_fornecedores():
	try:
		clear_output(wait=True)
		exibir_cabecalho_sistema(titulo = "🕹️ GERENCIAMENTO DE FORNECEDORES 🎮")
		with sqlite3.connect(db) as conexao:
			cursor = conexao.cursor()
			cursor.execute("SELECT * FROM fornecedores")
			fornecedores = cursor.fetchall()

			if fornecedores:
				colunas = [
					("ID", "yellow", "center"),
					("Nome", "yellow", "left"),
					("CNPJ", "yellow", "left"),
					("Telefone", "yellow", "right")
				]
				exibir_tabela(colunas, fornecedores)
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()
			else:
				print("Nenhum fornecedor encontrado!")
				time.sleep(2)
	except sqlite3.Error as erro:
		print(f"Erro ao listar os fornecedores: {erro}")
		time.sleep(2)

In [ ]:
def listar_reposicoes():
	try:
		clear_output(wait=True)
		exibir_cabecalho_sistema(titulo = "🕹️ GERENCIAMENTO DE REPOSIÇÕES 🎮")
		with sqlite3.connect(db) as conexao:
			cursor = conexao.cursor()
			cursor.execute("""
				SELECT
					r.id,
					p.nome AS nome_produto,
					f.nome AS nome_fornecedor,
					r.quantidade,
					r.data_reposicao
				FROM reposicoes r
				JOIN produtos p ON r.produto_id = p.id
				JOIN fornecedores f ON r.fornecedor_id = f.id
				ORDER BY r.data_reposicao DESC
			""")
			reposicoes = cursor.fetchall()

			if reposicoes:
				colunas = [
					("ID", "yellow", "center"),
					("Produto", "yellow", "left"),
					("Fornecedor", "yellow", "left"),
					("Quantidade", "", "right"),
					("Data da Reposição", "yellow", "center")
				]

				exibir_tabela(colunas, reposicoes)
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()
			else:
				console.print("Nenhuma reposição registrada.", style="yellow")
				time.sleep(3)
	except sqlite3.Error as erro:
		console.print("Erro ao listar reposições", erro)
		time.sleep(3)

#ATUALIZAR

In [ ]:
def atualizar_reposicao():
  try:
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()
      id = validar_reposicao_id()
      nova_quantidade = validar_estoque_reposicao()
      nova_data = validar_data()

      cursor.execute('''UPDATE reposicoes
        SET quantidade = ?
        WHERE id = ?''', (nova_quantidade, id))

      if nova_data is not None:
        cursor.execute('''UPDATE reposicoes
          SET data_reposicao = ?
          WHERE id = ?''', (nova_data, id))

      if cursor.rowcount == 0:
        print('[bright_red]Nenhuma reposição foi atualizado![/bright_red]')
        time.sleep(2)
      else:
          print('[bright_green]reposição atualizado com sucesso![/bright_green]')
          time.sleep(2)


  except sqlite3.Error as erro:
    print(f'[bright_red]Erro ao atualizar: {erro}[/bright_red]')
    time.sleep(2)
  except ValueError as erro:
    print(f'[bright_red]Erro ao atualizar: {erro}[/bright_red]')
    time.sleep(2)

In [ ]:
def atualizar_fornecedor():
  try:
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()

      novo_cnpj = None

      id = validar_fornecedor_id()
      novo_nome = input("Digite o novo nome do fornecedor(Aperte Enter para manter): ").title()
      if novo_nome == '':
        novo_nome = None
      else:
        novo_cnpj = validar_cnpj()



      novo_telefone = input("Digite o novo telefone do fornecedor(Aperte Enter para não cadastrar): ")
      if novo_telefone == '':
        novo_telefone = None

      if novo_nome is not None:
        cursor.execute('''UPDATE fornecedores
          SET nome = ?
          WHERE id = ?''', (novo_nome, id))

      if novo_cnpj is not None:
        cursor.execute('''UPDATE fornecedores
          SET cnpj = ?
          WHERE id = ?''', (novo_cnpj, id))

      if novo_telefone is not None:
        cursor.execute('''UPDATE fornecedores
          SET telefone = ?
          WHERE id = ?''', (novo_telefone, id))

      if cursor.rowcount == 0:
        print('[bright_red]Nenhum fornecedor foi atualizado![/bright_red]')
        time.sleep(2)
      else:
          print('[bright_green]Fornecedor atualizado com sucesso![/bright_green]')
          time.sleep(2)


  except sqlite3.Error as erro:
    print(f'[bright_red]Erro ao atualizar: {erro}[/bright_red]')
    time.sleep(2)


In [ ]:
def atualizar_produto():
  try:
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()

      id = validar_produto_id()
      teste = True
      teste2 = True

      while teste:
        nova_quantidade = input("Digite a nova quantidade do produto(aperte Enter para manter): ")
        if nova_quantidade == '':
          nova_quantidade = None
          teste = False

        else:
          nova_quantidade = int(nova_quantidade)

          if nova_quantidade < 0:
            print('[bright_red]Quantidade inválida![/bright_red]')
            time.sleep(2)

          else:
            teste = False

      while teste2:
        novo_preco = input("Digite o novo preço do produto(Aperte Enter para manter): ")

        if novo_preco == '':
          novo_preco = None
          teste2 = False
        else:
          novo_preco = float(novo_preco)
          if novo_preco <= 0:
            print('[bright_red]Preço inválido![/bright_red]')
            time.sleep(2)
          else:
            teste2 = False

      cursor.execute('''SELECT preco FROM produtos WHERE id = ?''', (id,))
      preco_antigo = cursor.fetchone()[0]

      if novo_preco is not None:
        cursor.execute('''UPDATE produtos
          SET preco = ?
          WHERE id = ?''', (novo_preco, id))

      if nova_quantidade is not None:
        cursor.execute('''UPDATE produtos
          SET quantidade_estoque = ?
          WHERE id = ?''', (nova_quantidade, id))



      data_atualizacao = datetime.date.today()

      if novo_preco is not None:
        cursor.execute('''INSERT INTO historico_precos (id_produto, preco_antigo, preco_novo, data_alteracao)
        VALUES (?, ?, ?, ?)''', (id, preco_antigo, novo_preco, data_atualizacao))

      if cursor.rowcount == 0:
        print('[bright_red]Nenhum produto foi atualizado![/bright_red]')
        time.sleep(2)
      else:
          print('[bright_green]produto atualizado com sucesso![/bright_green]')
          time.sleep(2)


  except sqlite3.Error as erro:
    print(f'[bright_red]Erro ao atualizar: {erro}[/bright_red]')
    time.sleep(2)
  except ValueError as erro:
    print(f'[bright_red]Erro ao atualizar: {erro}[/bright_red]')
    time.sleep(2)


#DELETAR

In [ ]:
def confirmar_exclusao(tipo: str, nome: str) -> bool:
    texto = f"Tem certeza que deseja excluir {tipo} [bold yellow]{nome}[/]?"
    return Confirm.ask(texto)

def deletar_produto():
    listar_produtos()

    id_do_produto = Prompt.ask("\nID do produto a excluir")
    if not id_do_produto.isdigit():
        console.print("[bright_red]ID inválido[/]")
        return

    with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()
        cursor.execute("SELECT nome FROM produtos WHERE id = ?", (id_do_produto,))
        nome_do_produto = cursor.fetchone()
        if not nome_do_produto:
            console.print("[bright_red]Produto não encontrado[/]")
            time.sleep(2)
            return

        if confirmar_exclusao("o produto", nome_do_produto[0]):
            cursor.execute("DELETE FROM produtos WHERE id = ?", (id_do_produto,))
            conexao.commit()
            console.print("[bright_green]Produto removido com sucesso![/]")
            time.sleep(2)



In [ ]:
def deletar_fornecedor():
    listar_fornecedores()

    id_fornecedor = Prompt.ask("\nID do fornecedor a excluir")
    if not id_fornecedor.isdigit():
        console.print("[bright red]ID inválido[/]")
        time.sleep(2)
        return

    with sqlite3.connect(db) as conexao:
        cursor = conexao.cursor()

        cursor.execute("SELECT nome FROM fornecedores WHERE id = ?", (id_fornecedor,))
        nome_do_fornecedor = cursor.fetchone()
        if not nome_do_fornecedor:
            console.print("[yellow]Fornecedor não encontrado[/yellow]")
            time.sleep(2)
            return

        cursor.execute("SELECT COUNT(*) FROM produtos WHERE fornecedor_id = ?", (id_fornecedor,))
        quantidade_de_produtos = cursor.fetchone()[0]
        if quantidade_de_produtos > 0:
            console.print(f"[bright red]Fornecedor possui {quantidade_de_produtos} produto(s) vinculado(s).[/]")
            console.print("Exclua ou transfira os produtos antes, ou ative ON DELETE CASCADE.")
            time.sleep(5)
            return

        if confirmar_exclusao("o fornecedor", nome_do_fornecedor[0]):
            cursor.execute("DELETE FROM fornecedores WHERE id = ?", (id_fornecedor,))
            conexao.commit()
            console.print("[bright green]Fornecedor removido com sucesso![/]")
            time.sleep(2)


#RELATÓRIOS

In [ ]:
def buscar_produto_ou_fornecedor(buscar):
	try:
		clear_output(wait=True)
		exibir_cabecalho_sistema(titulo = "🕹️ BUSCA POR PRODUTO OU FORNECEDOR 🎮")
		with sqlite3.connect(db) as conexao:
			cursor = conexao.cursor()
			cursor.execute("""
				SELECT
					p.id,
					p.nome,
					p.preco,
					p.categoria,
					p.quantidade_estoque,
					IFNULL((
						SELECT r.quantidade
						FROM reposicoes r
						WHERE r.produto_id = p.id
						ORDER BY r.data_reposicao DESC
						LIMIT 1
					), 0) AS ultima_reposicao,
					f.nome AS fornecedor
				FROM produtos p
				JOIN fornecedores f ON p.fornecedor_id = f.id
				WHERE p.nome LIKE ? OR f.nome LIKE ?
			""", (f"%{buscar}%", f"%{buscar}%"))

			resultados = cursor.fetchall()

			if resultados:
				colunas = [
					("ID", "yellow", "center"),
					("Produto", "yellow", "left"),
					("Preço", "yellow", "right"),
					("Categoria", "yellow", "left"),
					("Quantidade", "yellow", "center"),
					("Estoque", "yellow", "center"),
					("Fornecedor", "yellow", "left")
				]
				exibir_tabela(colunas, resultados)
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()
			else:
				console.print("Produto ou Fornecedor não encontrado.", style="bright_red")
				time.sleep(3)

	except sqlite3.Error as erro:
		console.print("Erro ao buscar produtos ou fornecedor", erro)
		time.sleep(3)

In [ ]:
def listar_historico_precos():
	try:
		clear_output(wait=True)
		exibir_cabecalho_sistema(titulo = "🕹️ HISTÓRICO DE PREÇOS 🎮")
		with sqlite3.connect(db) as conexao:
			cursor = conexao.cursor()
			cursor.execute('''
				SELECT historico_precos.id_produto,
				       produtos.nome,
				       historico_precos.preco_antigo,
				       historico_precos.preco_novo,
				       historico_precos.data_alteracao
				FROM historico_precos
				JOIN produtos ON historico_precos.id_produto = produtos.id
				ORDER BY data_alteracao DESC
			''')

			historico_bruto = cursor.fetchall()

			if historico_bruto:
				historico_formatado = []

				for id_prod, nome, preco_antigo, preco_novo, data in historico_bruto:
					variacao = ((preco_novo - preco_antigo) / preco_antigo) * 100
					variacao_str = f"{variacao:+.2f}%"


					cor = "red" if variacao > 0 else "green" # vermelho para alta, verde para queda
					variacao_texto = Text(variacao_str, style=cor)

					historico_formatado.append((
						str(id_prod),
						nome,
						f"R$ {preco_antigo:.2f}",
						f"R$ {preco_novo:.2f}",
						data,
						variacao_texto
					))

				colunas = [
					("ID Produto", "yellow", "center"),
					("Nome", "yellow", "left"),
					("Preço Antigo", "yellow", "right"),
					("Preço Novo", "red", "right"),
					("Data", "yellow", "center"),
					("Variação(%)", "yellow", "right")  # cor individual já aplicada no conteúdo
				]
				exibir_tabela(colunas, historico_formatado)
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()
			else:
				console.print("Nenhuma alteração de preço registrada.", style="bright_red")
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()

	except sqlite3.Error as erro:
		console.print("Erro ao listar histórico de preços", erro)

In [ ]:
def menor_estoque():
	try:
		clear_output(wait=True)
		exibir_cabecalho_sistema(titulo="🕹️ PRODUTOS COM MENOR ESTOQUE 🎮")
		with sqlite3.connect(db) as conexao:
			cursor = conexao.cursor()
			cursor.execute("""
				SELECT * FROM produtos
				ORDER BY quantidade_estoque ASC
				LIMIT 5
			""")
			produtos = cursor.fetchall()
			if produtos:
				colunas = [
					("ID", "bright_yellow", "center"),
					("Nome", "bright_yellow", "left"),
					("Preço", "bright_yellow", "right"),
					("Categoria", "bright_yellow", "left"),
					("Quantidade", "bright_yellow", "right"),
					("Fornecedor", "bright_yellow", "left")
				]

				produtos_coloridos = []
				for p in produtos:
					linha = list(p)
					quantidade = p[4]
					linha[4] = f"[bold red]{quantidade}[/bold red]" if quantidade < 5 else str(quantidade)
					produtos_coloridos.append(linha)

				exibir_tabela(colunas, produtos_coloridos)
				time.sleep(2)
				console.print("[turquoise2]Pressione Enter para continuar...[/]")
				input()
			else:
				console.print("Nenhum produto encontrado!", style="yellow")
				time.sleep(3)
	except sqlite3.Error as erro:
		console.print(f"Erro ao gerar relatório: {erro}", style="red")
		time.sleep(3)

In [ ]:
def relatorio_fonecedores_5():
  try:
    clear_output(wait=True)
    exibir_cabecalho_sistema(titulo="🕹️ RELATÓRIO DOS FORNECEDORES 🎮")
    with sqlite3.connect(db) as conexao:
      cursor = conexao.cursor()
      cursor.execute("""
      SELECT F.nome, COUNT(fornecedor_id) AS total_reposicao
      FROM fornecedores F
      JOIN reposicoes R
      ON F.id = R.fornecedor_id
      GROUP BY F.nome, F.id
      ORDER BY total_reposicao DESC
      LIMIT 5;
      """)
      produtos = cursor.fetchall()
      if produtos:
        colunas = [
                  ("Fornecedor", "yellow", "center"),
                  ("Total de Reposições", "yellow", "left")
                  ]

      exibir_tabela(colunas,produtos)
      time.sleep(2)
      console.print("[turquoise2]Pressione Enter para continuar...[/]")
      input()
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

#MENUS

In [ ]:
def exibir_tela_apresentacao_game_store_dinamica_v3_vermelha():

    #Exibe uma tela de apresentação "Bem-vindo à Game Store" em vermelho com um efeito
    #de pulsação no título e uma barra de progresso.

    console.clear() # Limpa o terminal para uma exibição limpa

    # Título principal
    titulo_base_text = Text(
        "🕹️ GAMES STORE 👾",
        justify="center",
        style="bold bright_red reverse", # Estilo base: negrito, vermelho brilhante, invertido
    )

    # Subtítulo (aparecerá after the main animation)
    subtitulo_base = "Onde a aventura começa."

    # Panel for the main title
    panel_principal = Panel(
        Align.center(titulo_base_text),
        expand=True,
        border_style="red",
        padding=(3, 10),
    )

    # --- Panel Animation ---
    with Live(panel_principal, console=console, screen=True, refresh_per_second=10) as live:
        # Efeito de Pulsação/Onda no Título
        pulsing_styles = ["bold bright_red reverse", "bold bright_red", "bold red reverse", "bold red"]
        for _ in range(5): # Pulsa 5 vezes
            for style in pulsing_styles + pulsing_styles[::-1]: # Vai e volta nos estilos
                titulo_pulsante = Text(
                    "🕹️ GAMES STORE 👾",
                    justify="center",
                    style=style
                )
                panel_principal.renderable = Align.center(titulo_pulsante)
                live.update(panel_principal)
                time.sleep(0.1) # Velocidade da pulsação

        time.sleep(1) # Pausa after the panel animation

    console.clear() # Clear the screen after the panel animation

    # --- Progress Bar Animation ---
    with Progress(
        SpinnerColumn("dots", style="bright_red"), # Spinner vermelho
        TextColumn("[progress.description]{task.description}", style="red"),
        BarColumn(bar_width=None, style="bright_red", complete_style="green", finished_style="red"), # Barra vermelha
        TextColumn("[progress.percentage]{task.percentage:>3.0f}%", style="red"),
        console=console,
        transient=True # Desaparece ao final
    ) as progress:
        task = progress.add_task("[bold red]Carregando aventuras...", total=100)

        while not progress.finished:
            progress.update(task, advance=1)
            time.sleep(0.04) # Velocidade da barra de progresso

    time.sleep(0.5) # Pausa final before printing final messages

    # --- Final Messages ---
    console.print(subtitulo_base, justify="center", style="italic yellow") # Print subtitle after animations
    console.print(Text("Bem-vindo(a) ao campo de batalha da diversão!", style="bold bright_red", justify="center"))
    console.print(Text("Que seus pixels sejam gloriosos!", style="italic yellow", justify="center"))

    time.sleep(3.5)

In [ ]:
def exibir_cabecalho_sistema(titulo="Sistema de Controle de Estoque"):
	cabecalho = Text(titulo, style="bold white on red", justify="center")
	panel = Panel(
		Align.center(cabecalho),
		border_style="bold red",
		title="📦 GAME STORE",
		title_align="left",
		padding=(1, 2)
	)
	console.print(panel)

In [ ]:
def exibir_tabela(colunas, dados):
	tabela = Table(
		#title=titulo,
		pad_edge=True,
		box=box.SQUARE,
		expand=True,  # orça ocupar toda a largura do console
		show_lines= True,
		style="bold red"
	)

	for nome_coluna, estilo, alinhamento in colunas:
		tabela.add_column(
			nome_coluna,
			style=estilo,
			justify=alinhamento,
			# Sem 'width': o Rich calcula com base no conteúdo
			no_wrap=False  # permite quebra de linha se necessário
		)

	for linha in dados:
		tabela.add_row(*[item if isinstance(item, Text) else str(item) for item in linha])

	console.print(tabela)

In [ ]:
def exibir_tela_finalizacao_game_store_dinamica_v3_vermelha():
    time.sleep(0.5)

    clear_output() # Limpa o terminal para uma exibição limpa

    # Título principal
    titulo_base_text = Text(
        "🕹️ GAMES STORE 👾",
        justify="center",
        style="bold bright_red reverse", # Estilo base: negrito, vermelho brilhante, invertido
    )

    # Subtítulo
    subtitulo_base = """A GAMES STORE agradece, boa sorte em suas aventuras!
    Produzido por
    © 2025 Grupo 7, Inc. Todos os direitos reservados."""

    # Panel for the main title
    panel_principal = Panel(
        Align.center(titulo_base_text),
        expand=True,
        border_style="red",
        padding=(3, 10),
    )

    # --- Panel Animation ---
    with Live(panel_principal, console=console, screen=True, refresh_per_second=10) as live:
        # Efeito de Pulsação/Onda no Título
        pulsing_styles = ["bold bright_red reverse", "bold bright_red", "bold red reverse", "bold red"]
        for _ in range(3): # Pulsa 3 vezes
            for style in pulsing_styles + pulsing_styles[::-1]:
                titulo_pulsante = Text(
                    "🕹️ GAMES STORE 👾",
                    justify="center",
                    style=style
                )
                panel_principal.renderable = Align.center(titulo_pulsante)
                live.update(panel_principal)
                time.sleep(0.1) # Velocidade da pulsação

        time.sleep(1) # Pausa after the panel animation

    console.clear() # Clear the screen after the panel animation

    # --- Progress Bar Animation ---
    with Progress(
        SpinnerColumn("dots", style="bright_red"), # Spinner vermelho
        TextColumn("[progress.description]{task.description}", style="red")
    ) as progress:
        task = progress.add_task("[bold red]Finalizando...", total=100)


        while not progress.finished:
            progress.update(task, advance=1)
            time.sleep(0.04)

    clear_output() # Limpa o terminal antes da mensagem final

    # --- Final Messages ---
    console.print(subtitulo_base, justify="center", style="italic yellow")
    time.sleep(3)

In [ ]:
def pedir_opc():
  try:
    time.sleep(1)
    opcao = Prompt.ask("Digite aqui")
    return opcao
  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

In [ ]:
def menu_reposicao():
  try:
    while True:

      clear_output(wait=True)
      titulo = "🕹️ GERENCIAMENTO DE REPOSIÇÃO 🎮"
      exibir_cabecalho_sistema(titulo)
      colunas = [
                ("Digito", "cyan", "center"),
                ("Selecione uma opçao", "bright_yellow", "center")
      ]

      dados = [
              ("1", "CADASTRAR NOVA REPOSIÇÃO"),
              ("2", "ATUALIZAR REPOSIÇÃO"),
              ("3", "LISTAR REPOSIÇÕES"),
              ("4", "VOLTAR")
      ]

      exibir_tabela(colunas, dados)

      opcao = pedir_opc()

      if opcao == '1':
        inserir_reposicao()
      elif opcao == '2':
        atualizar_reposicao()
      elif opcao == '3':
        listar_reposicoes()
      elif opcao == '4':
        break
      else:
        print("Opção inválida. Tente novamente.")


  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

  except ValueError as erro:
    print(f'[bright_red]Apenas números -> {erro}[/bright_red]')

  print('[orange_red1]voltando...[/]')
  time.sleep(2)

In [ ]:
def menu_fornecedor():
  try:
    while True:

      clear_output(wait=True)

      titulo = "🕹️ GERENCIAMENTO DE FORNECEDORES 🎮"
      exibir_cabecalho_sistema(titulo)
      colunas = [
                ("Digito", "cyan", "center"),
                ("Selecione uma opçao", "bright_yellow", "center")
      ]

      dados = [
              ("1", "CADASTRAR NOVO FORNECEDOR"),
              ("2", "ATUALIZAR FORNECEDOR"),
              ("3", "LISTAR FORNECEDORES"),
              ("4", "DELETAR FORNECEDOR"),
              ("5", "VOLTAR")
      ]

      exibir_tabela(colunas, dados)

      opcao = pedir_opc()

      if opcao == '1':
        cadastrar_fornecedores()
      elif opcao == '2':
        atualizar_fornecedor()
      elif opcao == '3':
        listar_fornecedores()
      elif opcao == '4':
        deletar_fornecedor()
      elif opcao == '5':
        break
      else:
        print("Opção inválida. Tente novamente.")


  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

  except ValueError as erro:
    print(f'[bright_red]Apenas números -> {erro}[/bright_red]')

  print('[orange_red1]voltando...[/]')
  time.sleep(2)

In [ ]:
def menu_produtos():
  try:
    while True:

      clear_output(wait=True)

      titulo = "🕹️ GERENCIAMENTO DE PRODUTO 🎮"
      exibir_cabecalho_sistema(titulo)
      colunas = [
                ("Digito", "cyan", "center"),
                ("Selecione uma opçao", "bright_yellow", "center")
      ]

      dados = [
              ("1", "CADASTRAR NOVOS PRODUTOS"),
              ("2", "ATUALIZAR PRODUTOS"),
              ("3", "LISTAR PRODUTOS"),
              ("4", "DELETAR PRODUTOS"),
              ("5", "VOLTAR")
      ]

      exibir_tabela(colunas, dados)

      opcao = pedir_opc()

      if opcao == '1':
        cadastrar_produtos()
      elif opcao == '2':
        atualizar_produto()
      elif opcao == '3':
        listar_produtos()
      elif opcao == '4':
        deletar_produto()
      elif opcao == '5':
        break
      else:
        print("Opção inválida. Tente novamente.")


  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

  except ValueError as erro:
    print(f'[bright_red]Apenas números -> {erro}[/bright_red]')

  print('[orange_red1]voltando...[/]')
  time.sleep(2)

In [ ]:
def menu_relatorios():
  try:
    while True:

      clear_output(wait=True)

      titulo = "🕹️ RELATÓRIOS 🎮"
      exibir_cabecalho_sistema(titulo)
      colunas = [
                ("Digito", "cyan", "center"),
                ("Selecione uma opçao", "bright_yellow", "center")
      ]

      dados = [
              ("1", "HISTÓRICO DE PREÇOS"),
              ("2", "RELATORIO DOS FORNECEDORES"),
              ("3", "RELATORIO DOS MENORES ESTOQUES"),
              ("4", "BUSCAR POR NOME"),
              ("5", "BUSCAR POR FORNECEDOR"),
              ("6", "VOLTAR")
      ]

      exibir_tabela(colunas, dados)

      opcao = pedir_opc()

      if opcao == '1':
        listar_historico_precos()
      elif opcao == '2':
        relatorio_fonecedores_5()
      elif opcao == '3':
        menor_estoque()
      elif opcao == '4':
        busca_nome = input('Digite o nome do produto: ').title()
        buscar_produto_ou_fornecedor(busca_nome)
      elif opcao == '5':
        busca_fornecedor = input('Digite o nome do fornecedor: ').title()
        buscar_produto_ou_fornecedor(busca_fornecedor)
      elif opcao == '6':
        break


      else:
        print("Opção inválida. Tente novamente.")


  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuario -> {erro}[/bright_red]')

  except ValueError as erro:
    print(f'[bright_red]Apenas números -> {erro}[/bright_red]')

  print('[orange_red1]voltando...[/]')
  time.sleep(2)

In [ ]:
def menu():
  exibir_tela_apresentacao_game_store_dinamica_v3_vermelha()
  try:
    while True:
      clear_output(wait=True)
      exibir_cabecalho_sistema(titulo="🕹️ SISTEMA DE CONTROLE DE ESTOQUE 🎮")

      if verificar_alerta_estoque_baixo():
        console.print(Align.center(Text("🚨 Atenção! Há produtos com estoque baixo 🚨", style="bold bright_yellow")))

      colunas = [
        ("Digito", "cyan", "center"),
        ("Selecione uma opção", "bright_yellow", "center")
      ]

      dados = [
        ("1", "GERENCIAMENTO DE PRODUTO"),
        ("2", "GERENCIAMENTO DE FORNECEDORES"),
        ("3", "GERENCIAMENTO DE REPOSIÇÕES"),
        ("4", "RELATÓRIOS"),
        ("5", "SAIR")
      ]

      exibir_tabela(colunas, dados)

      opcao = pedir_opc()

      if opcao == '1':
        menu_produtos()
      elif opcao == '2':
        menu_fornecedor()
      elif opcao == '3':
        menu_reposicao()
      elif opcao == '4':
        menu_relatorios()
      elif opcao == '5':
        break
      else:
        print("Opção inválida. Tente novamente.")
        time.sleep(2)

  except KeyboardInterrupt as erro:
    print(f'[bright_red]Interrupção do usuário -> {erro}[/bright_red]')
  except ValueError as erro:
    print(f'[bright_red]Apenas números disponíveis nas opções -> {erro}[/bright_red]')

  exibir_tela_finalizacao_game_store_dinamica_v3_vermelha()

# INICIAR

In [ ]:
menu()

                               A GAMES STORE agradece, boa sorte em suas aventuras!                                
                                                    Produzido por                                                  
                                  © 2025 Grupo 7, Inc. Todos os direitos reservados.                               